##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# マルチワーカー CPU/GPU トレーニングを移行する

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/multi_worker_cpu_gpu_training"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/guide/migrate/multi_worker_cpu_gpu_training.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/guide/migrate/multi_worker_cpu_gpu_training.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/guide/migrate/multi_worker_cpu_gpu_training.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a>   </td>
</table>

このガイドでは、マルチワーカーの分散トレーニングワークフローを TensorFlow 1 から TensorFlow 2 に移行する方法を実演します。

CPU/GPU を使用してマルチワーカートレーニングを実行するには

- TensorFlow 1 では、`tf.estimator.train_and_evaluate` および `tf.estimator.Estimator` API を使用してきました。
- TensorFlow 2 では、Keras API を使用して、モデル、損失関数、オプティマイザー、および指標を記述します。次に、Keras `Model.fit` API またはカスタムトレーニングループ（`tf.GradientTape`）を使用し、`tf.distribute.experimental.ParameterServerStrategy` または `tf.distribute.MultiWorkerMirroredStrategy` を使用して複数のワーカーにトレーニングを分散します。詳細については、次のチュートリアルを参照してください。
    - [TensorFlow による分散型トレーニング](../../guide/distributed_training.ipynb)
    - [Keras Model.fit/カスタムトレーニングループを使用したパラメータサーバートレーニング](../../tutorials/distribute/parameter_server_training.ipynb)
    - [Keras Model.fit を使用した MultiWorkerMirroredStrategy](../../tutorials/distribute/multi_worker_with_keras.ipynb)
    - [カスタムトレーニングループを使用した MultiWorkerMirroredStrategy](../../tutorials/distribute/multi_worker_with_ctl.ipynb)。

## セットアップ

必要とされるインポートとデモ用の単純なデータセットから始めます。

In [ ]:
# The notebook uses a dataset instance for `Model.fit` with
# `ParameterServerStrategy`, which depends on symbols in TF 2.7.
# Install a utility needed for this demonstration
!pip install portpicker

import tensorflow as tf
import tensorflow.compat.v1 as tf1

In [ ]:
features = [[1., 1.5], [2., 2.5], [3., 3.5]]
labels = [[0.3], [0.5], [0.7]]
eval_features = [[4., 4.5], [5., 5.5], [6., 6.5]]
eval_labels = [[0.8], [0.9], [1.]]

TensorFlow で複数のマシンでトレーニングするには、`'TF_CONFIG'` 構成環境変数が必要になります。`'TF_CONFIG'` を使用して、`'cluster'` と `'task'` のアドレスを指定します。（詳しくは [Distributed_training](../...guide/distributed_training.ipynb) ガイドを参照してください）。

In [ ]:
import json
import os

tf_config = {
    'cluster': {
        'chief': ['localhost:11111'],
        'worker': ['localhost:12345', 'localhost:23456', 'localhost:21212'],
        'ps': ['localhost:12121', 'localhost:13131'],
    },
    'task': {'type': 'chief', 'index': 0}
}

os.environ['TF_CONFIG'] = json.dumps(tf_config)

注意: 残念ながら、TensorFlow 1 の `tf.estimator` API を使用したマルチワーカートレーニングには複数のクライアントが必要になるため、（この Colab ノートブックでこれを実行するのが特に難しくなります）、ローカルトレーニングにフォールバックするように `'TF_CONFIG'` 環境変数なしでノートブックが実行できるようにします。（詳細については、[TensorFlow を使用した分散トレーニング](../../guide/distributed_training.ipynb) ガイドの <em data-md-type="emphasis">`'TF_CONFIG'` 環境変数の設定</em>セクションを参照してください。）

`del` ステートメントを使用して変数を削除します（ただし、TensorFlow 1 での実際のマルチワーカートレーニングでは、これを行う必要はありません）。

In [ ]:
del os.environ['TF_CONFIG']

## TensorFlow 1: tf.estimator API を使用したマルチワーカー分散トレーニング

次のコードスニペットは、TF1 でのマルチワーカートレーニングの正規のワークフローを示しています。`tf.estimator.Estimator`、`tf.estimator.TrainSpec`、`tf.estimator.EvalSpec`、およびトレーニングを分散する `tf.estimator.train_and_evaluate` API を使用します。

In [ ]:
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices((features, labels)).batch(1)

def _eval_input_fn():
  return tf1.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

def _model_fn(features, labels, mode):
  logits = tf1.layers.Dense(1)(features)
  loss = tf1.losses.mean_squared_error(labels=labels, predictions=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())
  return tf1.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

estimator = tf1.estimator.Estimator(model_fn=_model_fn)
train_spec = tf1.estimator.TrainSpec(input_fn=_input_fn)
eval_spec = tf1.estimator.EvalSpec(input_fn=_eval_input_fn)
tf1.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

## TensorFlow 2: 分散ストラテジーによるマルチワーカートレーニング

TensorFlow 2 では、`tf.distribute.Strategy` を介して、CPU、GPU、および TPU を使用する複数のワーカーでの分散トレーニングが行われます。

次の例は、2 つのストラテジー `tf.distribute.experimental.ParameterServerStrategy` と `tf.distribute.MultiWorkerMirroredStrategy` の使用方法を示しています。どちらも複数のワーカーによる CPU/GPU トレーニングのために設計されています。

`ParameterServerStrategy` は、*コーディネーター*（`'chief'`）を採用しているため、この Colab ノートブックの環境により使いやすくなっています。ここでは、いくつかのユーティリティを使用して、実行可能なエクスペリエンスに不可欠なサポート要素をセットアップします。スレッドを使用してパラメータサーバー（`'ps'`）とワーカー（`'worker'`）をシミュレートする *インプロセスクラスタ*を作成します。パラメータサーバーのトレーニングの詳細については、[ParameterServerStrategy を使用したパラメータサーバーのトレーニング](../../tutorials/distribute/parameter_server_training.ipynb)のチュートリアルを参照してください。

この例では、まず `'TF_CONFIG'` 環境変数を `tf.distribute.cluster_resolver.TFConfigClusterResolver` で定義して、クラスター情報を提供します。分散トレーニングにクラスター管理システムを使用している場合は、すでに `'TF_CONFIG'` が提供されているかどうかを確認してください。提供されている場合、この環境変数を明示的に設定する必要はありません。（詳細については、[TensorFlow を使用した分散トレーニング](../../guide/distributed_training.ipynb)ガイドの <em data-md-type="emphasis">`'TF_CONFIG'` 環境変数の設定</em>セクションを参照してください。）

In [ ]:
# Find ports that are available for the `'chief'` (the coordinator),
# `'worker'`s, and `'ps'` (parameter servers).
import portpicker

chief_port = portpicker.pick_unused_port()
worker_ports = [portpicker.pick_unused_port() for _ in range(3)]
ps_ports = [portpicker.pick_unused_port() for _ in range(2)]

# Dump the cluster information to `'TF_CONFIG'`.
tf_config = {
    'cluster': {
        'chief': ["localhost:%s" % chief_port],
        'worker': ["localhost:%s" % port for port in worker_ports],
        'ps':  ["localhost:%s" % port for port in ps_ports],
    },
    'task': {'type': 'chief', 'index': 0}
}
os.environ['TF_CONFIG'] = json.dumps(tf_config)

# Use a cluster resolver to bridge the information to the strategy created below.
cluster_resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver()

次に、ワーカーとパラメータサーバーの `tf.distribute.Server` を 1 つずつ作成します。

In [ ]:
# Workers need some inter_ops threads to work properly.
# This is only needed for this notebook to demo. Real servers
# should not need this.
worker_config = tf.compat.v1.ConfigProto()
worker_config.inter_op_parallelism_threads = 4

for i in range(3):
  tf.distribute.Server(
      cluster_resolver.cluster_spec(),
      job_name="worker",
      task_index=i,
      config=worker_config)

for i in range(2):
  tf.distribute.Server(
      cluster_resolver.cluster_spec(),
      job_name="ps",
      task_index=i)

実際の分散トレーニングでは、コーディネーターですべての `tf.distribute.Server` を開始せずに、複数のマシンを使用し、`"worker"` と `"ps"`（パラメータサーバー）は、それぞれ `tf.distribute.Server` を実行します。詳細については、[パラメータサーバのトレーニング](../../tutorials/distribute/parameter_server_training.ipynb) チュートリアルの*実世界のクラスタ*セクションを参照してください。

すべての準備が整ったら、`ParameterServerStrategy` オブジェクトを作成します。

In [ ]:
strategy = tf.distribute.experimental.ParameterServerStrategy(cluster_resolver)

ストラテジーオブジェクトを作成したら、モデル、オプティマイザ、およびその他の変数を定義し、`Strategy.scope` API 内で Keras `Model.compile` を呼び出してトレーニングを分散します。（詳細については、`Strategy.scope` API ドキュメントを参照してください。）

例えば、フォワードパスとバックワードパスを定義してトレーニングをカスタマイズする場合は、詳細について、[パラメータサーバートレーニング](../../tutorials/distribute/parameter_server_training.ipynb)チュートリアルの*カスタムトレーニングループを使用したトレーニング*セクションを参照してください。

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(
      (features, labels)).shuffle(10).repeat().batch(64)

eval_dataset = tf.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).repeat().batch(1)

with strategy.scope():
  model = tf.keras.models.Sequential([tf.keras.layers.Dense(1)])
  optimizer = tf.keras.optimizers.legacy.Adagrad(learning_rate=0.05)
  model.compile(optimizer, "mse")

model.fit(dataset, epochs=5, steps_per_epoch=10)

In [ ]:
model.evaluate(eval_dataset, steps=10, return_dict=True)

> **パーティショナ（`tf.distribute.experimental.practitioners{/code 0}）`**
>
> TensorFlow 2 の `ParameterServerStrategy` は、変数のパーティショニングをサポートし、TensorFlow 1 と同じパーティショナを提供しますが、紛らわしい名前はありません。
>
> - `tf.compat.v1.variable_axis_size_partitioner` -&gt; `tf.distribute.experimental.partitioners.MaxSizePartitioner`: シャードを最大サイズ以下に保つパーティショナ）。
> - `tf.compat.v1.min_max_variable_partitioner` -&gt; `tf.distribute.experimental.partitioners.MinSizePartitioner`: シャードごとに最小サイズを割り当てるパーティショナ。
> - `tf.compat.v1.fixed_size_partitioner` -&gt; `tf.distribute.experimental.partitioners.FixedShardsPartitioner`: 固定数のシャードを割り当てるパーティショナ。

または、`MultiWorkerMirroredStrategy` オブジェクトを使用できます。

In [ ]:
# To clean up the `TF_CONFIG` used for `ParameterServerStrategy`.
del os.environ['TF_CONFIG']
strategy = tf.distribute.MultiWorkerMirroredStrategy()

上記で使用したストラテジーを `MultiWorkerMirroredStrategy` オブジェクトに置き換えて、このストラテジーでトレーニングを実行できます。

`tf.estimator` API と同様に、`MultiWorkerMirroredStrategy` はマルチクライアントストラテジーであるため、この Colab ノートブックでは簡単に分散トレーニングを実行できません。したがって、上記のコードをこのストラテジーに置き換えると、ローカルで実行されることになります。マルチワーカートレーニング [Keras Model.fit を使用](../../tutorials/distribute/multi_worker_with_keras.ipynb)/[カスタムトレーニングループ](../../tutorials/distribute/multi_worker_with_ctl.ipynb)のチュートリアルでは、Colab のローカルホストで 2 つのワーカーを使用して変数を設定し、`'TF_CONFIG'` を使用してマルチワーカートレーニングを実行する方法を示しています。実際には、外部 IP アドレス/ポートに複数のワーカーを作成し、`'TF_CONFIG'` 変数を使用して各ワーカーのクラスター構成を指定します。

## 次のステップ

TensorFlow 2 の `tf.distribute.experimental.ParameterServerStrategy` と `tf.distribute.MultiWorkerMirroredStrategy` を使用したマルチワーカー分散トレーニングの詳細については、次のリソースを参照してください。

- チュートリアル: [ParameterServerStrategy と Keras Model.fit/カスタムトレーニングループを使用したパラメータサーバートレーニング](../../tutorials/distribute/parameter_server_training.ipynb)
- チュートリアル: [MultiWorkerMirroredStrategy と Keras Model.fit を使用したマルチワーカートレーニング](../../tutorials/distribute/multi_worker_with_keras.ipynb)
- チュートリアル: [MultiWorkerMirroredStrategy とカスタムトレーニングループを使用したマルチワーカートレーニング](../../tutorials/distribute/multi_worker_with_ctl.ipynb)
- ガイド: [TensorFlow による分散トレーニング](../../guide/distributed_training.ipynb)
- ガイド: [TensorFlow Profiler を使用した TensorFlow GPU パフォーマンスの最適化](../../guide/gpu_performance_analysis.ipynb)
- ガイド: [GPU を使用する](../../guide/gpu.ipynb)（複数の GPU を使用するセクション）